In [37]:
import warnings
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import Button, Layout
from datetime import datetime
# warnings.filterwarnings("ignore")
from umap import UMAP
import re
import pandas as pd
from skorch.dataset import Dataset
import matplotlib.patches as mpatches
from pprint import pprint
import numpy as np
from ipywidgets import HBox, VBox, interactive
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from sklearn import metrics
from IPython.display import display, Markdown
import ipywidgets as widgets
import math
from sklearn.manifold import TSNE
import torch
from ipywidgets import interact, interact_manual
from IPython.display import set_matplotlib_formats
from importlib import reload
from pathlib import Path
from tqdm import tqdm

import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))

import model_codebase as cb
import cicids2017 as cicids2017
import data_generator as generator

reload(cicids2017)
reload(generator)

pd.set_option('display.max_rows', 500)
pd.set_option('float_format', '{:.4f}'.format)

RES_PATH = "."
MODELPATH = Path(f"../res/{RES_PATH}/ts2vec.torch")
DATASETPATH_CACHE = Path(f"../res/{RES_PATH}/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")

In [29]:
df_res = pd.DataFrame()

sigma_values = [-.5, -.25, -.05, .0, .05, .25, .5]

model_args = {
    "input_size": 19,
    "rnn_size": 128,
    "rnn_layers": 3,
    "latent_size": 128
}

ts2vec = cb.STC(**model_args).eval()
ts2vec.load_state_dict(torch.load(str(MODELPATH), map_location=torch.device('cpu')))

testing_attacks = cicids2017.load_dataset(DATASETPATH_CACHE)["testing_attacks"]
testing_attacks = Dataset(*cb.dataset2tensors(testing_attacks))

for sigma in tqdm(sigma_values):
    ts2vec.sigma = sigma
    sigma_res = { "sigma": sigma }

    # Detection capabilities ..... #
    X_attacks = testing_attacks.X["context"]
    y_attacks = testing_attacks.y
    y_hat = ts2vec.context_anomaly(X_attacks).detach()
    y_hat = np.round(y_hat.cpu())
    
    metrics_rep = [ metrics.roc_auc_score,
                    metrics.precision_score, metrics.recall_score,
                    metrics.accuracy_score, metrics.f1_score]
    for m in metrics_rep:
        mres = m(y_attacks, y_hat)
        sigma_res[m.__name__] = mres
    
    tn, fp, fn, tp = metrics.confusion_matrix(y_attacks, y_hat, normalize="all").ravel()
    sigma_res.update({ "tn": tn, "fp": fp, "fn": fn, "tp": tp })
    sigma_res = pd.DataFrame(sigma_res, index=[0])
    df_res = pd.concat([df_res, sigma_res], ignore_index=True)

100%|██████████| 7/7 [00:35<00:00,  5.08s/it]


In [41]:
best_results = df_res.sort_values("roc_auc_score", ascending=False)
latex_str = best_results.to_latex(index=False, float_format="%.3f")
print(re.sub(' +',' ', latex_str))

\begin{tabular}{rrrrrrrrrr}
\toprule
 sigma & roc\_auc\_score & precision\_score & recall\_score & accuracy\_score & f1\_score & tn & fp & fn & tp \\
\midrule
-0.250 & 0.845 & 0.903 & 0.700 & 0.952 & 0.789 & 0.862 & 0.010 & 0.038 & 0.090 \\
-0.050 & 0.842 & 0.613 & 0.753 & 0.908 & 0.676 & 0.811 & 0.061 & 0.032 & 0.096 \\
 0.000 & 0.829 & 0.500 & 0.772 & 0.872 & 0.607 & 0.773 & 0.099 & 0.029 & 0.099 \\
-0.500 & 0.819 & 0.966 & 0.640 & 0.951 & 0.770 & 0.869 & 0.003 & 0.046 & 0.082 \\
 0.050 & 0.810 & 0.406 & 0.790 & 0.825 & 0.536 & 0.724 & 0.148 & 0.027 & 0.101 \\
 0.250 & 0.655 & 0.181 & 0.921 & 0.458 & 0.303 & 0.340 & 0.532 & 0.010 & 0.118 \\
 0.500 & 0.500 & 0.128 & 1.000 & 0.128 & 0.227 & 0.000 & 0.872 & 0.000 & 0.128 \\
\bottomrule
\end{tabular}

